In [67]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import marqo
import pprint

In [68]:
df = pd.read_csv('./data/filevine_case_study_justice - train set_source data set.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          2697 non-null   int64 
 1   ID                  2697 non-null   int64 
 2   name                2697 non-null   object
 3   href                2697 non-null   object
 4   docket              2687 non-null   object
 5   term                2697 non-null   object
 6   first_party         2696 non-null   object
 7   second_party        2696 non-null   object
 8   facts               2697 non-null   object
 9   facts_len           2697 non-null   int64 
 10  majority_vote       2697 non-null   int64 
 11  minority_vote       2697 non-null   int64 
 12  first_party_winner  2685 non-null   object
 13  decision_type       2692 non-null   object
 14  disposition         2638 non-null   object
 15  issue_area          2584 non-null   object
dtypes: int64(5), object(11)


In [69]:
mq = marqo.Client(url="http://localhost:8882")

In [70]:
query = df['facts'][0]
print(query)
results = mq.index("test").search(query, search_method='TENSOR')

# Prints the nicely formatted dictionary
pprint.pprint(results)

<p>On October 10, 1980, an Oklahoma City police officer shot and killed Albert Tuttle outside a bar. Rose Marie Tuttle, Albert’s widow, sued the police officer and the city in district court under Section 1983 of the Civil Rights Act of 1871, which allows an individual to recover damages against a party who “acting under color of state law” deprives another of his constitutional rights. The district court instructed the jury that the city could be held liable only if the incident had been caused by a municipal “policy,” but a single, unusually excessive use of force could support a finding that the city was grossly negligent or deliberately indifferent in the training or supervision of its police force and was therefore liable under Section 1983. The jury returned a verdict in favor of the police officer but against the city and awarded Tuttle’s estate $1.5 million in damages. The U.S. Court of Appeals for the Tenth Circuit affirmed.</p>



{'hits': [{'_highlights': [{'facts_clean': 'on october 10 1980 an oklahoma '
                                           'city police officer shot and '
                                           'killed albert tuttle outside a bar '
                                           'rose marie tuttle alberts widow '
                                           'sued the police officer and the '
                                           'city in district court under '
                                           'section 1983 of the civil rights '
                                           'act of 1871 which allows an '
                                           'individual to recover damages '
                                           'against a party who acting under '
                                           'color of state law deprives '
                                           'another of his constitutional '
                                           'rights the district court '
      

In [71]:
df['href'][0]

'https://api.oyez.org/cases/1984/83-1919'

In [72]:
results_df = pd.DataFrame(results['hits'])
same_doc_index = results_df[results_df['href']==df['href'][0]].index
results_df.drop(same_doc_index , inplace=True)
most_similar_document = results_df.loc[results_df['_score'].idxmax()]

In [73]:
results_df

,facts_clean,href,_id,_highlights,_score
1,alberta jefferson an african american woman di...,https://api.oyez.org/cases/1997/96-957,8d46051d-7c10-4bd5-b93a-8eb59fb3bb61,[{'facts_clean': 'alberta jefferson an african...,0.871906
2,in the early morning hours of new years eve 20...,https://api.oyez.org/cases/2013/13-551,b26b2102-f5d5-413b-bc9b-18e16fd25af9,[{'facts_clean': 'in the early morning hours o...,0.863521
3,the los angeles county sheriffs department obt...,https://api.oyez.org/cases/2006/06-605,1858911e-24bc-4dfd-bff0-5a8406031c73,[{'facts_clean': 'the los angeles county sheri...,0.861175
4,tucson police officer andrew kisela and two ot...,https://api.oyez.org/cases/2017/17-467,354badac-d6f1-4504-9486-ed7e998735cb,[{'facts_clean': 'tucson police officer andrew...,0.860328
5,mary lu redmond a former police officer receiv...,https://api.oyez.org/cases/1995/95-266,a461c0e5-12a8-4fb9-aa22-94b6d1c21a54,[{'facts_clean': 'mary lu redmond a former pol...,0.859716
6,in 1975 eight chicano individuals were attendi...,https://api.oyez.org/cases/1985/85-224,e0e312d8-bf28-4fdb-973f-dd1065564720,[{'facts_clean': 'in 1975 eight chicano indivi...,0.859048
7,tracey l johnson and david james jr were polic...,https://api.oyez.org/cases/2014/13-1318,15935a61-a079-455c-946e-805e34216e69,[{'facts_clean': 'tracey l johnson and david j...,0.857731
8,during the civil rights movement of the 1960s ...,https://api.oyez.org/cases/1963/39,fbd76eb7-e3a5-456d-b933-b03e89473b09,[{'facts_clean': 'during the civil rights move...,0.855161
9,a monroe county court sentenced walter mcmilli...,https://api.oyez.org/cases/1996/96-542,2242fd3a-249b-4077-a7e5-638a454be77f,[{'facts_clean': 'a monroe county court senten...,0.854289


In [74]:
most_similar_document

facts_clean    alberta jefferson an african american woman di...
href                      https://api.oyez.org/cases/1997/96-957
_id                         8d46051d-7c10-4bd5-b93a-8eb59fb3bb61
_highlights    [{'facts_clean': 'alberta jefferson an african...
_score                                                  0.871906
Name: 1, dtype: object

In [75]:
mq.index("test").get_document(
    document_id=most_similar_document['_id'],
    expose_facets=True
)

{'facts_clean': 'alberta jefferson an african american woman died as a result of a fire in her home in the city of tarrant alabama her survivors filed multiple complaints against tarrant city two under state law and two under federal law the state law complaints alleged wrongful death and the commonlaw tort of outrage while the two federal claims brought under 42 usc section 1983 alleged that ms jeffersons death was the direct result of indifference and racial discrimination in violation of the fourteenth amendments due process and equal protection clauses the city claimed that the complaints were governed by alabamas wrongful death act which the alabama supreme court had interpreted to provide only for punitive damages the city then argued that it could not be sued under section 1983 because the supreme court had ruled that section 1983 plaintiffs are not entitled to sue a municipality for punitive damages\nthe state court ruled in favor of jefferson but the alabama supreme court reve

In [76]:
cond = df['href'] =='https://api.oyez.org/cases/1997/96-957'

result = df[cond].facts.values[0]

result

"<p>Alberta Jefferson, an African American woman, died as a result of a fire in her home in the city of Tarrant, Alabama. Her survivors filed multiple complaints against Tarrant City: two under state law and two under federal law. The state law complaints alleged wrongful death and the common-law tort of outrage, while the two federal claims brought under 42 U.S.C. Section 1983 alleged that Ms. Jefferson's death was the direct result of indifference and racial discrimination in violation of the Fourteenth Amendment's Due Process and Equal Protection Clauses. The City claimed that the complaints were governed by Alabama's Wrongful Death Act, which the Alabama Supreme Court had interpreted to provide only for punitive damages. The City then argued that it could not be sued under Section 1983 because the Supreme Court had ruled that Section 1983 plaintiffs are not entitled to sue a municipality for punitive damages.</p>\n<p>The state court ruled in favor of Jefferson, but the Alabama Supr

In [77]:
cond = df['href'] =='https://api.oyez.org/cases/1997/96-957'

query = df[cond].facts.values[0]
print(query)
results = mq.index("sentences_test").search(query, search_method='TENSOR')

# Prints the nicely formatted dictionary
pprint.pprint(results)

<p>Alberta Jefferson, an African American woman, died as a result of a fire in her home in the city of Tarrant, Alabama. Her survivors filed multiple complaints against Tarrant City: two under state law and two under federal law. The state law complaints alleged wrongful death and the common-law tort of outrage, while the two federal claims brought under 42 U.S.C. Section 1983 alleged that Ms. Jefferson's death was the direct result of indifference and racial discrimination in violation of the Fourteenth Amendment's Due Process and Equal Protection Clauses. The City claimed that the complaints were governed by Alabama's Wrongful Death Act, which the Alabama Supreme Court had interpreted to provide only for punitive damages. The City then argued that it could not be sued under Section 1983 because the Supreme Court had ruled that Section 1983 plaintiffs are not entitled to sue a municipality for punitive damages.</p>
<p>The state court ruled in favor of Jefferson, but the Alabama Suprem

In [78]:
results_df = pd.DataFrame(results['hits'])
same_doc_index = results_df[results_df['href']=='https://api.oyez.org/cases/1997/96-957'].index
results_df.drop(same_doc_index , inplace=True)
most_similar_document = results_df.loc[results_df['_score'].idxmax()]

In [79]:
results_df

,facts_clean,href,_id,_highlights,_score
1,on october 10 1980 an oklahoma city police off...,https://api.oyez.org/cases/1984/83-1919,cfda625a-45f8-4f14-8a1c-7becf9cf14d3,[{'facts_clean': 'on october 10 1980 an oklaho...,0.871481
2,as a result of a lawsuit in 1974 the jefferson...,https://api.oyez.org/cases/1988/87-1614,dbaad204-9da7-436e-8bdf-56bfca47ae46,[{'facts_clean': 'as a result of a lawsuit in ...,0.868458
3,bonnie weisgram died of carbon monoxide poison...,https://api.oyez.org/cases/1999/99-161,a0d2b9b1-c30c-4be1-b2e4-74dbf10d09ee,[{'facts_clean': 'bonnie weisgram died of carb...,0.868359
4,a georgia state court convicted lawrence jeffe...,https://api.oyez.org/cases/2009/09-8852,87f53911-6436-4d19-9d5d-d67ee1836bc0,[{'facts_clean': 'a georgia state court convic...,0.865187
5,in 1986 timothy tyrone foster an 18yearold bla...,https://api.oyez.org/cases/2015/14-8349,96c59bda-bb92-4a12-bfe1-3f8002ccaab6,[{'facts_clean': 'in 1986 timothy tyrone foste...,0.864633
6,millerel alleged the prosecution in his capita...,https://api.oyez.org/cases/2004/03-9659,43c9176f-3595-456c-89b0-5e0dbdb3266f,[{'facts_clean': 'millerel alleged the prosecu...,0.859979
7,roy heck was convicted of voluntary manslaught...,https://api.oyez.org/cases/1993/93-6188,0fd8efa1-c7ef-4db8-be21-3e7f19c7c796,[{'facts_clean': 'roy heck was convicted of vo...,0.858869
8,after a car accident in south carolina hanna b...,https://api.oyez.org/cases/1964/171,9b1a10e0-b6eb-4c13-ac90-05ec37e6c9bc,[{'facts_clean': 'after a car accident in sout...,0.858750
9,on december 30 1984 james mcwilliams raped and...,https://api.oyez.org/cases/2016/16-5294,5fde4c76-81f9-4b31-b878-9544d5dd64b3,[{'facts_clean': 'on december 30 1984 james mc...,0.858294


In [80]:
mq.index("sentences_test").get_document(
    document_id=most_similar_document['_id'],
    expose_facets=True
)

{'facts_clean': 'on october 10 1980 an oklahoma city police officer shot and killed albert tuttle outside a bar rose marie tuttle alberts widow sued the police officer and the city in district court under section 1983 of the civil rights act of 1871 which allows an individual to recover damages against a party who acting under color of state law deprives another of his constitutional rights the district court instructed the jury that the city could be held liable only if the incident had been caused by a municipal policy but a single unusually excessive use of force could support a finding that the city was grossly negligent or deliberately indifferent in the training or supervision of its police force and was therefore liable under section 1983 the jury returned a verdict in favor of the police officer but against the city and awarded tuttles estate 15 million in damages the us court of appeals for the tenth circuit affirmed',
 'href': 'https://api.oyez.org/cases/1984/83-1919',
 '_id'